In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight : bold;}
div.input {font-family:Consolas; font-size:12pt;}
</style>
"""))

<b><font color = "red" size = "6">ch10. 데이터베이스 연동</font></b>

# 1절. SQLite 데이터베이스 연결
- SQLite 데이터베이스는 별도의 DBMS 없이 SQL을 이용하여 DB에 엑세스할 수 있도록 만든 간단한 **디스크
기반의 데이터베이스를 제공**하는 C 라이브러리다
- SQLite는 프로토타입(시제품 나오기전까지의 테스트 제품)을 만들때 사용한다. 정식 제품은 오라클이나
다른 DBMS를 인식한다.
- [DB browser for SQLite](https://sqlitebrowser.org/dl)

## 1.1 SQLite browser 설치 및 패키지 load

In [2]:
import sqlite3

# sqlite 버전 확인
sqlite3.sqlite_version

'3.35.4'

In [4]:
import pandas as pd

# pandas 버전 확인
pd.__version__

'1.2.4'

## 1.2 데이터베이스 연결

In [5]:
# ▶ DB 연결 생성시, 파일이 있으면 연결, 파일이 없으면 빈 파일을 생성한다 ◀
conn = sqlite3.connect('Data/ch10_example.db')
conn

In [6]:
# ▶ 커서 객체반환. 커서는 SQL문을 실행시키고 결과데이터(SELECT)를 조회하는데 사용된다 ◀
cursor = conn.cursor()
cursor

In [7]:
# ▶ 테이블 생성하기 ◀
cursor.execute("""
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
""")

In [8]:
cursor.execute("DROP TABLE MEMBER")

In [9]:
cursor.execute("""
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
""")

In [10]:
# ▶ 테이블에 데이터 입력하기 ◀
cursor.execute("INSERT INTO MEMBER VALUES('마초', 20, 'hong@naver.com')")
cursor.execute("INSERT INTO MEMBER VALUES('서서', 22, 'seo@naver.com')")
cursor.execute("INSERT INTO MEMBER VALUES('주창', 24, 'joo@naver.com')")

In [11]:
# ▶ DML문장은 COMMIT이 필요하다, 반대 : conn.rollback() ◀
conn.commit()

In [12]:
# ▶ SELECT 실행결과는 cursor가 가리킨다 ◀
cursor.execute("SELECT * FROM MEMBER")

In [13]:
# ▶ SELECT 실행결과를 받는 함수 리스트 ◀
# 1. fetchall() - 결과를 모두 받을 때( 데이터는 튜플, 출력은 List형태 )
# 2. fetchone() - 결과를 한 행씩 받을 때 ( 튜플 )
# 3. fetchmany(n) - 결과를 n행 받을 때
print(cursor.fetchall())

[('마초', 20, 'hong@naver.com'), ('서서', 22, 'seo@naver.com'), ('주창', 24, 'joo@naver.com')]


In [ ]:
# ▶ 한번 소요된 커서 객체는 다시 fetch할 수 없다 ◀
print(cursor.fetchall())

In [14]:
cursor.execute("SELECT * FROM MEMBER")
members = cursor.fetchall()

members

[('마초', 20, 'hong@naver.com'),
 ('서서', 22, 'seo@naver.com'),
 ('주창', 24, 'joo@naver.com')]

In [15]:
for member in members:
    print(member)

('마초', 20, 'hong@naver.com')
('서서', 22, 'seo@naver.com')
('주창', 24, 'joo@naver.com')


In [21]:
cursor.execute("SELECT * FROM MEMBER")
member_list = []
while True:
    member = cursor.fetchone()
    if member is None:
        break
    member_list.append({'name':member[0], 'age':member[1], 'email':member[2]})
    print("이름 :", member[0], "\t나이 :", member[1], "\t메일 :", member[2])

이름 : 마초 	나이 : 20 	메일 : hong@naver.com
이름 : 서서 	나이 : 22 	메일 : seo@naver.com
이름 : 주창 	나이 : 24 	메일 : joo@naver.com


In [22]:
print(member_list)

[{'name': '마초', 'age': 20, 'email': 'hong@naver.com'}, {'name': '서서', 'age': 22, 'email': 'seo@naver.com'}, {'name': '주창', 'age': 24, 'email': 'joo@naver.com'}]


In [23]:
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE DESC")
for member in cursor.fetchmany(2): # 나이가 많은 사람 두명이 출력
    print(member[0], member[1], member[2])

주창 24 joo@naver.com
서서 22 seo@naver.com


In [24]:
cursor.close() # 생략 가능
conn.close() # 꼭 닫아야함

## 1.3 SQL 구문에 파라미터 사용하기
- 방법1.qmark
- 방법2.named(추천)

In [26]:
conn = sqlite3.connect('Data/ch10_example.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM MEMBER WHERE NAME IN('마초', '서서')")
cursor.fetchall()

[('마초', 20, 'hong@naver.com'), ('서서', 22, 'seo@naver.com')]

In [29]:
# ▶ 파라미터 사용하기 ( q(question?)mark ) ◀
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
# cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(?, ?)", ('마초', '장비'))
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(?, ?)", (name1, name2))
cursor.fetchall()

검색할 이름1 :마초
검색할 이름2 :장비


[('마초', 20, 'hong@naver.com')]

In [31]:
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
names = [name1, name2]

cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(?, ?)", names)
cursor.fetchall()

검색할 이름1 :마초
검색할 이름2 :장비


[('마초', 20, 'hong@naver.com')]

In [34]:
# ▶ 파라미터 사용하기 ( named ) ◀
name1 = input('검색할 이름 :')
cursor.execute("SELECT * FROM MEMBER WHERE NAME = :name", {'name':name1})
members = cursor.fetchall()

if len(members) == 0:
    print('해당 이름의 데이터가 없습니다.')
else:
    print(members)
    print(cursor.fetchall()) # 한 행도 출력 안된다.

검색할 이름 :장비
해당 이름의 데이터가 없습니다.


In [39]:
# ▶ MEMBER 테이블에 입력 ◀
# 사용자로부터 이름, 나이, 메일을 입력받아 DB에 INSERT하는 프로그램
while True:
    try:
        name = input("이름 입력 :")
        age = int(input("나이 입력 :"))
        mail = input("메일 입력 :")
    except ValueError as e:
        print('숫자를 0이상 입력하세요.')
        age = 18
        mail = input("메일 입력 :")

    cursor.execute("INSERT INTO MEMBER VALUES(:name, :age, :mail)",{'name':name, 'age':age,
                                                                    'mail':mail})
    try:
        exit = input('종료(Y/N) :')
        if exit == 'Y':
            break
        elif exit == 'N':
            continue
    except ValueError as e1:
        print('잘못된 문자를 입력')
conn.commit()

이름 입력 :가후
나이 입력 :28
메일 입력 :hu@daum.net
종료(Y/N) :Y


In [41]:
cursor.execute("SELECT * FROM MEMBER")
cursor.fetchall()

[('마초', 20, 'hong@naver.com'),
 ('서서', 22, 'seo@naver.com'),
 ('주창', 24, 'joo@naver.com'),
 ('장비', 25, 'jang@naver.com'),
 ('조조', 26, 'jojo@naver.com'),
 ('마등', 20, 'deung@naver.com'),
 ('유비', 21, 'bi@naver.com'),
 ('가후', 28, 'hu@daum.net')]

In [42]:
class User:
    def __init__(self, name, age, mail):
        self.name = name
        self.age = age
        self.mail = mail
        
    def __str__(self):
        return '이름:{:4}\t나이:{:3}\t메일:{}'.format(self.name, self.age, self.mail)
    
    # 객체[]를 딕셔너리{:}로 바꾸는 함수 
    def as_dict(self):
        return {'name':self.name, 'age':self.age, 'mail':self.mail}

try:
    name = input("이름 입력 : ")
    age = int(input("나이 입력 : "))
    mail = input("메일 입력 : ")
except ValueError as e:
    print('숫자를 0이상 입력하세요.')
    age = 18
    mail = input("메일 입력 :")

user = User(name, age, mail)
    
cursor.execute("INSERT INTO MEMBER VALUES(:name, :age, :mail)", user.as_dict())
conn.commit()

이름 입력 :장합
나이 입력 :25
메일 입력 :hab@naver.com


In [43]:
cursor.close()
conn.close()

# 2절. 오라클 데이터베이스 연결

In [44]:
# ▶ 모듈 불러오기 및 dsn 설정 ◀
import cx_Oracle
oracle_dsn = cx_Oracle.makedsn(host = 'localhost', port = 1521, sid = 'xe')
oracle_dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [45]:
# ▶ conn 설정 방법 1 ◀
# 에러 발생 시, VC_redist.x64.exe를 설치해야 한다.
conn = cx_Oracle.connect("scott", "tiger", dsn = oracle_dsn)
conn

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [46]:
conn.close()

In [47]:
# ▶ conn 설정 방법 2 ◀
conn = cx_Oracle.connect("scott", "tiger", "localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [48]:
cursor = conn.cursor()

In [49]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)

emp = cursor.fetchall()

In [50]:
for e in emp:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [53]:
cursor.execute(sql)
for e in cursor:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [54]:
sql = "SELECT * FROM EMP WHERE ENAME = :ename"
ename = input("검색할 이름 :")

cursor.execute(sql, {'ename':ename})
emp = cursor.fetchall()

if len(emp) == 0:
    print('입력하신 이름의 데이터는 없습니다')
else:
    for e in emp:
        print(e)

검색할 이름 :SMITH
(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)


In [61]:
# ▶ 데이터베이스 검색결과를 DataFrame으로 ◀
import pandas as pd

cursor.execute("SELECT * FROM EMP")
data = cursor.fetchall() # 튜플형식의 리스트로 출력
data_df = pd.DataFrame(data) # 데이터프레임

data_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [57]:
# ▶ 각 필드의 정보 (필드명, 타입, display_size, 정확도, scale, ...etc) ◀
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [58]:
[row[0] for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [59]:
data_df.columns

RangeIndex(start=0, stop=8, step=1)

In [62]:
data_df.columns = [row[0] for row in cursor.description]
data_df

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [63]:
data_df.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [64]:
cursor.close()
conn.close()

# 연습문제

In [ ]:
# 문제풀이형 1. (4)
# 문제풀이형 2. (2)